In [109]:
using Random, Distributions

== Finite State Markov Chain Approximation of AR(1) Process

For further details see Heer and Maussner, 2009 (p.222)

Consider the process:
\begin{eqnarray}
  z_{t+1} = \rho z_t + \epsilon_t, \hspace{1cm} \epsilon_t \sim N(0, \sigma^{2}_{\epsilon}) \nonumber
\end{eqnarray}

* Select the _size_ of the grid by choosing $\lambda \in R_{++}$ so that
  \begin{equation}
   z_1 = - \frac{\lambda \sigma_{\epsilon}}{\sqrt{1 - \rho^2}}
  \end{equation}
* Choose the number of grid points $m$
* Put  $step= - \frac{2 z_1}{(m-1)}$ and for $i=1, 2, \ldots , m $ compute $z_i = z_1 + (i-1) step$
* Compute the _transition matrix_ $P=(p_{ij})$. Let $\pi(\cdot)$ denote the _cumulative distribution function_ of the standard normal distribution. For $ i=1, 2, \ldots, m $ put
  \begin{equation}
     p_{i1} = \pi \left( \frac{z_1 - \rho z_i}{ \sigma_{\epsilon} }  + \frac{step}{2 \sigma_{\epsilon}} \right) \nonumber \\
     p_{ij} = \pi \left( \frac{z_j - \rho z_i}{\sigma_{\epsilon}} + \frac{step}{2 \sigma_{\epsilon}} \right) - \pi \left( \frac{z_j - \rho z_i}{\sigma_{\epsilon}} - \frac{step}{2 \sigma_{\epsilon}} \right) \mbox{ , } j=2,3,\ldots,m-1 \nonumber \\
     p_{im} = 1 - \Sigma^{m-1}_{j=1} p_{ij} \nonumber
  \end{equation}

The following Julia function takes $\rho$, $\sigma_{\epsilon}$, $\lambda$ and $m$ as input and returns the vector $z=[z_1, z_2, \ldots, z_m]'$ and the transition matrix $P$.


In [107]:
function fnTauchen(pRho, pSigmaEps, pLambda, pM)

    pSigmaZ = sqrt( pSigmaEps^2/(1-pRho^2) )
    zBar = pLambda*pSigmaZ
    zt = collect( -zBar:( 2*zBar/(pM-1) ):( -zBar+(pM-1)*( 2*zBar/(pM-1) ) ) )


    mP = ones(pM,pM)

    i = 1

    while i <= pM
    
        mP[i,1]   = Distributions.cdf( Normal(), (zt[1]-pRho*zt[i]+(zt[2]-zt[1])/2)/pSigmaEps ); 
    
        j = 2

        while j <= (pM-1)
     
            mP[i, j] =  Distributions.cdf(Normal(), (zt[j]-pRho*zt[i]+(zt[j]-zt[j-1])/2)/pSigmaEps ) - Distributions.cdf(Normal(), (zt[j]-pRho*zt[i]-(zt[j]-zt[j-1])/2)/pSigmaEps );
            j = j + 1
        
        end
        
        i = i + 1
    
    end


    mP[1:pM, pM]=mP[1:pM, pM]-sum( mP[1:pM,1:(pM-1)], dims=2 )

    return zt, mP
    
    
end

fnTauchen (generic function with 1 method)

In [108]:

zt, mP = fnTauchen(0.95, 0.005, 3, 4)

mP

4×4 Array{Float64,2}:
 0.996757     0.00324265   0.0         0.0
 0.000385933  0.998441     0.00117336  0.0
 1.73409e-21  0.00117336   0.998441    0.000385933
 1.04647e-54  3.51129e-20  0.00324265  0.996757